# "2019 September One Day Holding Long Short Portfolio BackTest"
> "2019 September Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [237]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [238]:
#hide_input
#df_tes=pd.read_csv('D:\\Downloads\\20194dayreturnpred2019train60K.csv')
df_test=pd.read_csv('D:\\Downloads\\sample28K20194dayreturnpred2019train60K.csv')
df_tes=pd.read_csv('D:\\Downloads\\10Q20194dayreturnpred2019train60K.csv')
#df_tes=pd.read_csv('D:\\Downloads\\1day60kvocab20192020testcumulative.csv')

In [239]:
#hide
df_tes['returnpred']=df_tes['labelr1']*df_tes['pred']
df_test['returnpred']=df_test['labelr1']*df_test['pred']

In [240]:
#hide
#df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20190101].loc[df_testp.Date<20190201].sort_values(by='Date',ascending=True).copy()
#df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190101].loc[df_test.Date<20190201].sort_values(by='Date',ascending=True).copy()
df_tes=df_tes.loc[df_tes.label!='nan'].loc[df_tes.Date>=20190901].loc[df_tes.Date<20191001].sort_values(by='Date',ascending=True).copy().reset_index(drop=True)
df_long=df_tes.loc[df_tes.pred==1].copy().reset_index(drop=True)
df_short=df_tes.loc[df_tes.pred==-1].copy().reset_index(drop=True)
#df_testp.reset_index(drop=True, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [241]:
#hide
#df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20190101].loc[df_testp.Date<20190201].sort_values(by='Date',ascending=True).copy()
#df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190101].loc[df_test.Date<20190201].sort_values(by='Date',ascending=True).copy()
df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190901].loc[df_test.Date<20191001].sort_values(by='Date',ascending=True).copy().reset_index(drop=True)
df_longa=df_test.loc[df_test.pred==1].copy().reset_index(drop=True)
df_shorta=df_test.loc[df_test.pred==-1].copy().reset_index(drop=True)
#df_testp.reset_index(drop=True, inplace=True)

In [242]:
#hide
df_tes['dailyaveragereturn']=0.0
for date in df_tes['Date'].unique():
    df_tes.loc[df_tes['Date']==date,'dailyaveragereturn']=np.prod(1+(df_tes.loc[df_tes['Date']==date]['returnpred']/100))**(1/len(df_tes.loc[df_tes['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

In [243]:
#hide
df_test['dailyaveragereturn']=0.0
for date in df_test['Date'].unique():
    df_test.loc[df_test['Date']==date,'dailyaveragereturn']=np.prod(1+(df_test.loc[df_test['Date']==date]['returnpred']/100))**(1/len(df_test.loc[df_test['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

In [244]:
#hide
df_a=df_tes.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)
df_b=df_test.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)

In [245]:
#hide
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1b']=np.prod(df_a['dailyaveragereturn'][:l+1])

df_b['cumreturn1c']=0.0
for l in range(len(df_b)):
    df_b.loc[l,'cumreturn1c']=np.prod(df_b['dailyaveragereturn'][:l+1])

In [246]:
#hide
df_a=pd.merge(df_a,df_b[['Date','cumreturn1c']], right_on='Date', left_on='Date',how='inner')

**Long-Short Portfolio**

In [247]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [248]:
#hide_input
df_a[['Date','cumreturn1b','cumreturn1c','cumreturndow','cumreturnsp500']]

,Date,cumreturn1b,cumreturn1c,cumreturndow,cumreturnsp500
0,2019-09-03,1.007236,1.008038,1.010842,1.009091
1,2019-09-04,0.970284,1.009434,1.023993,1.023361
2,2019-09-05,0.933025,1.008853,1.024925,1.026014
3,2019-09-06,0.911509,1.005206,1.024829,1.027471
4,2019-09-09,0.886074,1.008921,1.025159,1.030301
5,2019-09-10,0.764712,1.012175,1.032571,1.039016
6,2019-09-11,0.767546,1.012953,1.035544,1.040755
7,2019-09-12,0.732689,1.001991,1.034794,1.042174
8,2019-09-13,0.747834,0.999575,1.031549,1.036710
9,2019-09-16,0.784773,0.986052,1.034212,1.038011


In [249]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs 8KPortfolio (brown) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_a).mark_trail(color='brown').encode(
    x='Date:T',
    y='cumreturn1c:Q',
    size='cumreturn1c',

).properties(
    width=1000,
    height=1000
)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3 + a4# + a5

alt.LayerChart(...)

**Long portfolio**

In [250]:
#hide
df_long['dailyaveragereturn']=0.0
for date in df_long['Date'].unique():
    df_long.loc[df_long['Date']==date,'dailyaveragereturn']=np.prod(1+(df_long.loc[df_long['Date']==date]['returnpred']/100))**(1/len(df_long.loc[df_long['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_longa['dailyaveragereturn']=0.0
for date in df_longa['Date'].unique():
    df_longa.loc[df_longa['Date']==date,'dailyaveragereturn']=np.prod(1+(df_longa.loc[df_longa['Date']==date]['returnpred']/100))**(1/len(df_longa.loc[df_longa['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_a=df_long.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)
df_b=df_longa.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)

df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1b']=np.prod(df_a['dailyaveragereturn'][:l+1])
    
df_b['cumreturn1c']=0.0
for l in range(len(df_b)):
    df_b.loc[l,'cumreturn1c']=np.prod(df_b['dailyaveragereturn'][:l+1])

df_a=pd.merge(df_a,df_b[['Date','cumreturn1c']], right_on='Date', left_on='Date',how='inner')

In [251]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [252]:
#hide_input
df_a[['Date','cumreturn1b','cumreturn1c','cumreturndow','cumreturnsp500']]

,Date,cumreturn1b,cumreturn1c,cumreturndow,cumreturnsp500
0,2019-09-05,0.982801,1.052665,1.000911,1.002593
1,2019-09-09,1.011342,1.059115,1.001139,1.006782
2,2019-09-10,1.054167,1.079635,1.008377,1.015298
3,2019-09-11,1.030961,1.082320,1.008377,1.015298


In [253]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs 8KPortfolio (brown) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_a).mark_trail(color='brown').encode(
    x='Date:T',
    y='cumreturn1c:Q',
    size='cumreturn1c',

).properties(
    width=1000,
    height=1000
)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3 + a4# + a5

alt.LayerChart(...)

**Short Portfolio**

In [254]:
#hide
df_short['dailyaveragereturn']=0.0
for date in df_short['Date'].unique():
    df_short.loc[df_short['Date']==date,'dailyaveragereturn']=np.prod(1+(df_short.loc[df_short['Date']==date]['returnpred']/100))**(1/len(df_short.loc[df_short['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_shorta['dailyaveragereturn']=0.0
for date in df_shorta['Date'].unique():
    df_shorta.loc[df_shorta['Date']==date,'dailyaveragereturn']=np.prod(1+(df_shorta.loc[df_shorta['Date']==date]['returnpred']/100))**(1/len(df_shorta.loc[df_shorta['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_a=df_short.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)
df_b=df_shorta.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)

df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1b']=np.prod(df_a['dailyaveragereturn'][:l+1])
    
df_b['cumreturn1c']=0.0
for l in range(len(df_b)):
    df_b.loc[l,'cumreturn1c']=np.prod(df_b['dailyaveragereturn'][:l+1])

df_a=pd.merge(df_a,df_b[['Date','cumreturn1c']], right_on='Date', left_on='Date',how='inner')

In [255]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [256]:
#hide_input
df_a[['Date','cumreturn1b','cumreturn1c','cumreturndow','cumreturnsp500']]

,Date,cumreturn1b,cumreturn1c,cumreturndow,cumreturnsp500
0,2019-09-03,1.007236,1.002123,1.010842,1.009091
1,2019-09-04,0.970284,0.986861,1.023993,1.023361
2,2019-09-05,0.927952,0.984450,1.024925,1.026014
3,2019-09-06,0.906553,0.980806,1.024829,1.027471
4,2019-09-09,0.876254,0.980694,1.025159,1.030301
5,2019-09-10,0.728217,0.973774,1.032571,1.039016
6,2019-09-11,0.737269,0.973427,1.035544,1.040755
7,2019-09-12,0.703787,0.943833,1.034794,1.042174
8,2019-09-13,0.718333,0.938084,1.031549,1.036710
9,2019-09-16,0.753815,0.926150,1.034212,1.038011


In [257]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs 8KPortfolio (brown) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_a).mark_trail(color='brown').encode(
    x='Date:T',
    y='cumreturn1c:Q',
    size='cumreturn1c',

).properties(
    width=1000,
    height=1000
)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3 + a4# + a5

alt.LayerChart(...)